In [6]:
# importing all libraries needed 

import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import xml
import json
from pandas.io.json import json_normalize
import folium
# collecting website
url = requests.get ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

table_data = soup.find('table')
fields = table_data.find_all('td')
postcode = []
borough=[]
neighbourhood = []
for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
    
df_ca = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_ca.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_ca['Borough'].replace('Not assigned', np.nan, inplace = True)
df_ca.dropna(subset=['Borough'], inplace =True)
df_ca.head(10), df_ca.tail(10)    

df_ca_new= df_ca.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_ca_new.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_ca_new
    
df_ca = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_ca.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_ca.head(10), df_ca.tail(10)
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']
df_new = pd.merge(df_ca_new, df_geo, on = ['Postcode'], how = 'inner')
df_arrangement = df_new[[ 'Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()
df_arrangement2 = df_new[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df_arrangement2.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


CREATING MAP OF TORONTO

In [7]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print (latitude,longitude)

toronto_map = folium.Map(location =[latitude, longitude], zoom_start = 20)

#adding markers to map
for lat, lng, borough, neighbourhood in zip (df_arrangement['Latitude'], df_arrangement['Longitude'], df_arrangement['Borough'],
                                           df_arrangement['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    redius = 3,
    popup = label,
    color = 'green',
    fill= True,
    fill_color = '#3199cc',
    fill_opacity = 0.3,
    parse_html = False).add_to(toronto_map)
    
toronto_map

43.653963 -79.387207


In [9]:
df_downt = df_arrangement2[df_arrangement2['Borough'].str.contains('Toronto')]

to_data = df_downt.reset_index(drop = True)
to_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
